In [1]:
!pip install --upgrade pip
!pip install transformers
!pip install unidecode
!pip install git+https://github.com/deepset-ai/haystack.git#egg=farm-haystack[colab,faiss]

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 2.1 MB 34.3 MB/s 
  Attempting uninstall: pip
    Found existing installation: pip 21.1.3
    Uninstalling pip-21.1.3:
      Successfully uninstalled pip-21.1.3
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.4/4.4 MB 37.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 596.3/596.3 kB 12.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.5/101.5 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 55.0 MB/s eta 0:00:00
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
import pandas as pd
import numpy as np
import string
import unidecode
import os, glob, re, sys, random, unicodedata, collections

import matplotlib.pyplot as plt
import seaborn as sns
from wordcloud import WordCloud

plt.style.use('ggplot')

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,confusion_matrix,classification_report
from sklearn.feature_extraction.text import CountVectorizer

from tqdm import tqdm
import nltk
from nltk.sentiment import SentimentIntensityAnalyzer
from nltk.corpus import stopwords
from nltk.stem import RSLPStemmer
from nltk.tokenize import sent_tokenize , word_tokenize
nltk.download('words')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('maxent_ne_chunker')
nltk.download('vader_lexicon')
nltk.download('stopwords')
## criando lista de stopwords
STOP_WORDS = stopwords.words('portuguese') + list(string.punctuation)
STOP_WORDS.append('\n')

## Importação do BERT - Hugginface
from transformers import AutoModel, AutoTokenizer, AutoModelForPreTraining, BertTokenizer, BertForSequenceClassification, pipeline
import torch

import pathlib
from pathlib import Path

from haystack.utils import clean_wiki_text, convert_files_to_docs, fetch_archive_from_http, print_answers
from haystack.nodes import FARMReader, TransformersReader, DensePassageRetriever, PreProcessor
from haystack.document_stores import FAISSDocumentStore
from haystack.pipelines import ExtractiveQAPipeline

[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Unzipping corpora/words.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping chunkers/maxent_ne_chunker.zip.
[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [3]:
## Verificação da GPU utilizada
!nvidia-smi

Mon Jul  4 19:57:10 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   41C    P8     9W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [4]:
document_store = FAISSDocumentStore(faiss_index_factory_str="Flat")

INFO - haystack.telemetry -  Haystack sends anonymous usage data to understand the actual usage and steer dev efforts towards features that are most meaningful to users. You can opt-out at anytime by calling disable_telemetry() or by manually setting the environment variable HAYSTACK_TELEMETRY_ENABLED as described for different operating systems on the documentation page. More information at https://haystack.deepset.ai/guides/telemetry


In [5]:
# Montagem do Drive
from google.colab import drive
drive.mount('/content/drive')

work_path = '/content/drive/MyDrive/Classroom/PosGraduacao/ProjetoFinal/'
os.chdir(work_path)

Mounted at /content/drive


In [10]:
# Let's first get some files that we want to use
doc_dir = "livros"
# Convert files to dicts
docs = convert_files_to_docs(dir_path=doc_dir, clean_func=clean_wiki_text, split_paragraphs=True)

# Now, let's write the dicts containing documents to our DB.
document_store.write_documents(docs)

# apontando para o modelo treinado
nlp = pipeline("question-answering", model='pierreguillou/bert-base-cased-squad-v1.1-portuguese')

INFO - haystack.utils.preprocessing -  Converting livros/maoLuva.txt
INFO - haystack.utils.preprocessing -  Converting livros/quincas.txt
INFO - haystack.utils.preprocessing -  Converting livros/casaVelha.txt
INFO - haystack.utils.preprocessing -  Converting livros/esau.txt
INFO - haystack.utils.preprocessing -  Converting livros/memoriasBras.txt
INFO - haystack.utils.preprocessing -  Converting livros/domCasmurro.txt
INFO - haystack.utils.preprocessing -  Converting livros/iaia.txt
INFO - haystack.utils.preprocessing -  Converting livros/memorial-de-aires.txt
INFO - haystack.utils.preprocessing -  Converting livros/helena.txt
INFO - haystack.utils.preprocessing -  Converting livros/ressurreicao.txt


Writing Documents:   0%|          | 0/10 [00:00<?, ?it/s]

In [12]:
question = "Quem é Capitu?"

resultados = pd.DataFrame(columns=['Documento','Score'])
i = 0

for documento in tqdm(docs, desc='Livros'):
    result = nlp(question=question, context=documento.content)
    resultados.loc[i] = [documento.meta['name'], result['score']]
    i += 1
#resultados.head()
resultados.sort_values(by='Score', ascending=False).head()

Livros: 100%|██████████| 10/10 [1:05:18<00:00, 391.80s/it]


,Documento,Score
5,domCasmurro.txt,0.840419
4,memoriasBras.txt,0.744943
8,helena.txt,0.733090
2,casaVelha.txt,0.523793
1,quincas.txt,0.351217
